Install all requried libraries

In [8]:
!pip install -U torch torchvision
!pip install -U openai-whisper transformers spacy sentence-transformers
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Import Libraries and Load models

In [9]:
import os
import re
import spacy
from google.colab import files
from transformers import pipeline
import whisper

# Load all models once to avoid reloading them again and again.
uploaded = files.upload()
audio_file_name = next(iter(uploaded))

print("Loading all required models...")
try:
    # Load Whisper for transcription
    whisper_model = whisper.load_model("base")

    # Load model for classification
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Load model for summarization
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    # Load Spacy for location detection
    nlp = spacy.load("en_core_web_sm")

    print("All models loaded successfully.")
except Exception as e:
    print(f"An error occurred while loading models: {e}")

Saving call_124.mp3 to call_124.mp3
Loading all required models...


Device set to use cpu
Device set to use cpu


All models loaded successfully.


In [11]:
#Whisper Model to transcribe the text

model = whisper.load_model("base")
result = model.transcribe(audio_file_name, task="translate")
english_text = result["text"]
print(english_text)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Now I won't want to disturb, right, it's me. Yes, I have a gun and I don't, it's from Ali, I've loaded and some of this is a shot. I don't even know what happened. Okay, where are you? Say it one more time. Yes, it's as soon as you can. Is this an apartment or a house? Yes, I have a house. Yes, I have a house. Now I suppose I'll just call a place. Thank you. Please enjoy the music while your party is reached. Okay, this is a novel on upper here. What apartment number are you all in? Where is the person shot? Oh my god, oh my god. Where is the person shot? It's a female. Yes, it's a female. Miss Linda, get me a message for that please. Is someone shot in the chest? Okay, you can't form. Okay, now it's the inside or outside? No, I'm not. Is the inside, okay, listen, is the inside or outside? She's inside, she's inside. Is it a white female, black female? Honey, is it a white female or black female? Yes, it is white. Okay, what is her name? Yes, but it's actually... Okay, what is your na

In [ ]:
#Categorize the complaint into predefined crime categories

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
categories = ['Robbery', 'Assault', 'Cybercrime', 'Theft', 'Domestic Violence', 'Fraud', 'Missing Person', 'Harassment']
result = classifier(english_text, categories)
print("Category:", result["labels"][0])

Device set to use cpu


Category: Domestic Violence


In [ ]:
#Usecase 1 : Analyze the transcribed text to determine the level of danger or threat present in the person's speech.

high_threat = ["kill", "gun", "shoot", "bomb", "murder", "attack", "knife", "rape"]
medium_threat = ["beat", "threat", "follow", "scared", "stab", "danger"]
low_threat = ["scold", "argument", "push", "fight", "harass"]

def get_danger_score(text):
    text = text.lower()
    score = 1
    for word in high_threat:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            score += 2
    for word in medium_threat:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            score += 1
    for word in low_threat:
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            score += 0.5
    return min(5, round(score))

score = get_danger_score(english_text)
print("Danger Score:", score)

Danger Score: 5


In [ ]:
#Usecase 2 : Concise summary of the person's statement to quickly understand the context of the conversation.

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer(english_text, max_length=100, min_length=20, do_sample=False)
print(summary[0]['summary_text'])

Device set to use cpu


"Santa's the emergency" asks caller if he has a gun or a knife. "I want to get a really good description of this guy so we catch him" "He's wearing a giant sweater and a shark hat. A giant sweatshirt and an... What is it? Giant sweater and giant hat"


In [ ]:
#Usecase 3 : Extract the location mentioned by the person to identify where the incident
import numpy as np

nlp = spacy.load("en_core_web_sm")
fake_locations = {'Santa', 'Today', 'Tomorrow', 'Someone', 'Anything', 'Nobody'}

def extract_locations(text):
    doc = nlp(text)
    locations = []
    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC", "FAC"]:
            loc = ent.text.strip()
            if len(loc) > 3 and loc not in fake_locations and loc[0].isupper():
                locations.append(loc)
    return list(set(locations)) if locations else [np.nan]

locations = extract_locations(english_text)
print("Locations:", locations)

Locations: [nan]
